## Instacart Market Basket Analysis 

### a. Load the Data:
- Load the CSV files into pandas DataFrames.
- Check for missing values and data types.

In [1]:
import numpy as np
import pandas as pd

df_aisles = pd.read_csv('aisles.csv')
df_departments = pd.read_csv('departments.csv')
df_order_products_prior = pd.read_csv('order_products__prior.csv')
df_order_products_train = pd.read_csv('order_products__train.csv')
df_orders = pd.read_csv('orders.csv')
df_products = pd.read_csv('products.csv')


In [10]:

#test = df_order_products_prior.groupby('order_id').head()
count = 0
product_lists=[]
for user_id, group in df_order_products_prior.groupby('order_id'):
    if count > 10:
            break
    product_lists.append(list(group['product_id']))
    count = count + 1
        
print(product_lists)

[[33120, 28985, 9327, 45918, 30035, 17794, 40141, 1819, 43668], [33754, 24838, 17704, 21903, 17668, 46667, 17461, 32665], [46842, 26434, 39758, 27761, 10054, 21351, 22598, 34862, 40285, 17616, 25146, 32645, 41276], [13176, 15005, 47329, 27966, 23909, 48370, 13245, 9633, 27360, 6348, 40878, 6184, 48002, 20914, 37011, 12962, 45698, 24773, 18569, 41176, 48366, 47209, 46522, 38693, 48825, 8479], [40462, 15873, 41897], [34050, 46802], [23423], [21405, 47890, 11182, 2014, 29193, 34203, 14992, 31506, 23288, 44533, 18362, 27366, 432, 3990, 14183], [24852, 4796, 31717, 47766, 4605, 1529, 21137, 22122, 34134, 27156, 14992, 49235, 26842, 3464, 25720], [30162, 27085, 5994, 1313, 31506], [30597, 15221, 43772, 37886, 37215, 34335, 3164, 26910, 38888, 38050, 25213, 11175, 23390, 29471, 43511]]


 Check for missing values and data types

In [2]:
df_array = [
    df_aisles,
    df_departments,
    df_order_products_prior,
    df_order_products_train,
    df_orders,df_products
]

#Check if there is empty data
print("check empty values")
for df_iterate in df_array:
    print(df_iterate.empty)

#Check the shape
print("print shape")
for df_iterate in df_array:
    print(df_iterate.shape)

#Check the columns
print("print columns")
for df_iterate in df_array:
    print(df_iterate.columns)
    
#Check the first 5 rows
print("print first 5 rows")
for df_iterate in df_array:
    print(df_iterate.head())
    
#Check the last 5 rows
print("print last 5 rows")
for df_iterate in df_array:
    print(df_iterate.tail())
    
#Check the data types
print("print datatypes of columns")
for df_iterate in df_array:
    print(df_iterate.dtypes)
    
#Check info of dataframe
print("print info for dataframe")
for df_iterate in df_array:
    print(df_iterate.info())
    
#Check summary statistics of dataframe
print("print summary")
for df_iterate in df_array:
    print(df_iterate.describe())

#Check Nan values in dataframe
print("print number of NaN")
for df_iterate in df_array:
    print(df_iterate.isna().sum())

#Check null values in dataframe
print("print number of Null")
for df_iterate in df_array:
    print(df_iterate.isnull().sum())

#Check for duplicated rows in dataframe
print("print if there is a duplication")
for df_iterate in df_array:
    print(df_iterate.duplicated())

check empty values
False
False
False
False
False
False
print shape
(134, 2)
(21, 2)
(32434489, 4)
(1384617, 4)
(3421083, 7)
(49688, 4)
print columns
Index(['aisle_id', 'aisle'], dtype='object')
Index(['department_id', 'department'], dtype='object')
Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered'], dtype='object')
Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered'], dtype='object')
Index(['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order'],
      dtype='object')
Index(['product_id', 'product_name', 'aisle_id', 'department_id'], dtype='object')
print first 5 rows
   aisle_id                       aisle
0         1       prepared soups salads
1         2           specialty cheeses
2         3         energy granola bars
3         4               instant foods
4         5  marinades meat preparation
   department_id department
0              1     frozen
1              2      other
2        

### set the empty value in days_since_prior_order to -1 as it is the first order for each user

In [3]:
null_index = df_orders.loc[df_orders['days_since_prior_order'].isnull()].index
print(df_orders.loc[df_orders['days_since_prior_order'].isnull()].index)
df_orders.loc[df_orders['days_since_prior_order'].isnull(),'days_since_prior_order'] = 0
#remove user_id = 0 as all its data are 0
#i = df_orders[(df_orders.user_id == 0)].index
#df_orders.drop(i, inplace = True)

Index([      0,      11,      26,      39,      45,      50,      54,      75,
            79,      83,
       ...
       3420839, 3420863, 3420896, 3420919, 3420925, 3420930, 3420934, 3421002,
       3421019, 3421069],
      dtype='int64', length=206209)


### merge order_products prior and training tables

In [5]:
temp = df_order_products_train.copy()
temp.insert(0, 'istraining', 1)
df_order_products = df_order_products_prior.copy()
df_order_products.insert(0, 'istraining', 0)
frames = [df_order_products, temp]
df_order_products = pd.concat(frames, ignore_index=True)

In [6]:
#convert datatypes
#aisle
df_aisles['aisle_id'] = df_aisles['aisle_id'].astype('int64')
df_aisles['aisle'] = df_aisles['aisle'].astype('str')
#df_departments
df_departments['department_id'] = df_departments['department_id'].astype('int64')
df_departments['department'] = df_departments['department'].astype('str')
#df_products
df_products['product_id'] = df_products['product_id'].astype('int64')
df_products['product_name'] = df_products['product_name'].astype('str')
df_products['aisle_id'] = df_products['aisle_id'].astype('int64')
df_products['department_id'] = df_products['department_id'].astype('int64')
#df_order_products
df_order_products['product_id'] = df_order_products['product_id'].astype('int64')
df_order_products['order_id'] = df_order_products['order_id'].astype('int64')
df_order_products['add_to_cart_order'] = df_order_products['add_to_cart_order'].astype('int64')
df_order_products['reordered'] = df_order_products['reordered'].astype('bool')
df_order_products['istraining'] = df_order_products['istraining'].astype('bool')
#df_orders
df_orders['order_id'] = df_orders['order_id'].astype('int64')
df_orders['user_id'] = df_orders['user_id'].astype('int64')
df_orders['eval_set'] = df_orders['eval_set'].astype('str')
df_orders['order_number'] = df_orders['order_number'].astype('int64')
df_orders['order_dow'] = df_orders['order_dow'].astype('int64')
df_orders['order_hour_of_day'] = df_orders['order_hour_of_day'].astype('int64')
df_orders['days_since_prior_order'] = df_orders['days_since_prior_order'].astype('int64')

### c. Basic Descriptive Statistics:
- Calculate summary statistics for numerical features.
- Examine the distribution of categorical features.

In [8]:
table_names = [
    'aisles_table',
    'departments_table',
    'order_products_table',
    'orders_table',
    'products_table'
]
df_array = [
    df_aisles,
    df_departments,
    df_order_products,
    df_orders,
    df_products
]
i = 0
for df_iterate in df_array:
    print(f"For {table_names[i]} summary statistics for numerical features:")
    i += 1
    print(df_iterate.describe())

For aisles_table summary statistics for numerical features:
         aisle_id
count  134.000000
mean    67.500000
std     38.826537
min      1.000000
25%     34.250000
50%     67.500000
75%    100.750000
max    134.000000
For departments_table summary statistics for numerical features:
       department_id
count      21.000000
mean       11.000000
std         6.204837
min         1.000000
25%         6.000000
50%        11.000000
75%        16.000000
max        21.000000
For order_products_table summary statistics for numerical features:
           order_id    product_id  add_to_cart_order
count  3.381911e+07  3.381911e+07       3.381911e+07
mean   1.710566e+06  2.557551e+04       8.367738e+00
std    9.874008e+05  1.409770e+04       7.139540e+00
min    1.000000e+00  1.000000e+00       1.000000e+00
25%    8.554130e+05  1.351900e+04       3.000000e+00
50%    1.710660e+06  2.525600e+04       6.000000e+00
75%    2.565587e+06  3.793500e+04       1.100000e+01
max    3.421083e+06  4.968800e+0

- Examine the distribution of categorical features.

In [9]:
##For table order_products
#How many times a product is ordered
df_order_products.groupby('product_id').agg({'order_id':'count'})

#How many times the product is added first in cart
df_order_products[df_order_products.add_to_cart_order == 1].groupby('product_id').agg({
    'add_to_cart_order':'count'
})

##For table products
#How many category(aisle_id) for each department
df_products.groupby('department_id').agg({'aisle_id':'count'})

#How many product in each category(aisle_id)
df_products.groupby('aisle_id').agg({'product_id':'count'})

,product_id
aisle_id,
1,146
2,271
3,832
4,543
5,409
...,...
130,303
131,457
132,178


In [ ]:
##ChatGPT code
import pandas as pd
from datetime import timedelta

# Assuming df_orders is already defined

# Convert the 'order_dow' to a timedelta relative to '01/06/2018' for first orders
df_orders['date_of_order'] = pd.to_datetime('01/06/2018') + pd.to_timedelta(df_orders['order_dow'], unit='D')

# Calculate the cumulative sum of 'days_since_prior_order' for each user
df_orders['cumulative_days'] = df_orders.groupby('user_id')['days_since_prior_order'].cumsum()

# Adjust the date for subsequent orders
df_orders['date_of_order'] = df_orders['date_of_order'] + pd.to_timedelta(df_orders['cumulative_days'].fillna(0), unit='D')

# Drop the temporary 'cumulative_days' column if it's no longer needed
df_orders.drop(columns=['cumulative_days'], inplace=True)
